In [1]:
# Earth Engine and Common Libraries
import ee
from pathlib import Path

# Authenticate and initialize Earth Engine
try:
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')  # Try to use existing credentials first
except Exception:
    ee.Authenticate()
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

In [ ]:
import ee
import openforis_whisp as whisp
import geopandas as gpd
import pandas as pd
import time
import threading
from queue import Queue
import logging
from typing import List, Optional, Dict, Any
from concurrent.futures import ThreadPoolExecutor, as_completed

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("whisp-high-volume")

# Optimized configuration for EE high-volume processing
EE_MAX_CONCURRENT = 10
EE_FEATURES_PER_BATCH = 50  # Features per Earth Engine request
MAX_RETRIES = 3
THREAD_POOL_SIZE = 4

class OptimizedWhispProcessor:
    """Optimized processor using Earth Engine high-volume patterns"""
    
    def __init__(self, max_concurrent=EE_MAX_CONCURRENT, features_per_batch=EE_FEATURES_PER_BATCH):
        self.max_concurrent = max_concurrent
        self.features_per_batch = features_per_batch
        self.semaphore = threading.Semaphore(max_concurrent)
        self.results = {}
        self.processing_stats = {'completed': 0, 'failed': 0, 'total': 0}
        
    def process_file_optimized(self, geojson_path: str, national_codes: Optional[List[str]] = None) -> pd.DataFrame:
        """Process file using optimized Earth Engine batching"""
        
        # Load the GeoDataFrame
        gdf = gpd.read_file(geojson_path)
        total_features = len(gdf)
        
        logger.info(f"Processing {total_features} features in batches of {self.features_per_batch}")
        
        # Split into feature batches
        feature_batches = []
        for i in range(0, total_features, self.features_per_batch):
            batch = gdf.iloc[i:i+self.features_per_batch]
            feature_batches.append(batch)
        
        logger.info(f"Created {len(feature_batches)} batches for Earth Engine processing")
        
        # Process batches concurrently using ThreadPoolExecutor
        results = []
        with ThreadPoolExecutor(max_workers=self.max_concurrent) as executor:
            # Submit all batches
            future_to_batch = {
                executor.submit(self._process_feature_batch, batch, national_codes, i): i 
                for i, batch in enumerate(feature_batches)
            }
            
            # Collect results as they complete
            for future in as_completed(future_to_batch):
                batch_idx = future_to_batch[future]
                try:
                    batch_result = future.result()
                    results.append(batch_result)
                    logger.info(f"Completed batch {batch_idx + 1}/{len(feature_batches)}")
                except Exception as e:
                    logger.error(f"Error processing batch {batch_idx + 1}: {e}")
                    self.processing_stats['failed'] += 1
        
        # Combine all results
        if results:
            combined_df = pd.concat(results, ignore_index=True)
            self.processing_stats['completed'] += len(results)
            logger.info(f"Successfully processed {len(combined_df)} total features")
            return combined_df
        else:
            logger.warning("No results to combine")
            return pd.DataFrame()
    
    def _process_feature_batch(self, batch_gdf: gpd.GeoDataFrame, national_codes: Optional[List[str]], batch_idx: int) -> pd.DataFrame:
        """Process a single batch of features using Earth Engine"""
        
        with self.semaphore:  # Limit concurrent EE requests
            # Convert GeoDataFrame to Earth Engine FeatureCollection directly
            features_list = []
            
            for idx, row in batch_gdf.iterrows():
                # Convert each feature to EE format
                geometry = row.geometry
                properties = {k: v for k, v in row.items() if k != 'geometry' and pd.notna(v)}
                
                # Convert shapely geometry to EE geometry
                if geometry is not None:
                    ee_geometry = ee.Geometry(geometry.__geo_interface__)
                    ee_feature = ee.Feature(ee_geometry, properties)
                    features_list.append(ee_feature)
            
            # Create FeatureCollection from features
            feature_collection = ee.FeatureCollection(features_list)
            
            # Process using optimized Earth Engine parameters
            return self._process_ee_feature_collection(feature_collection, national_codes, batch_idx)
    
    # def _process_ee_feature_collection(self, feature_collection: ee.FeatureCollection, 
    #                                  national_codes: Optional[List[str]], batch_idx: int) -> pd.DataFrame:
    #     """Process FeatureCollection using Earth Engine with retry logic"""
        
    #     for attempt in range(MAX_RETRIES):
    #         try:
    #             # Use whisp's Earth Engine processing with optimized parameters
    #             result_fc = whisp.whisp_stats_ee_to_ee(
    #                 feature_collection=feature_collection,
    #                 external_id_column=None,
    #                 national_codes=national_codes
    #             )
                
    #             # Configure Earth Engine computation for high-volume processing
    #             # Use higher tileScale and maxPixels for distributed computation
    #             result_dict = result_fc.getInfo()  # This triggers the computation
                
    #             # Convert EE result back to DataFrame
    #             df_result = self._ee_dict_to_dataframe(result_dict)
                
    #             # Add ISO2 country codes
    #             df_result = whisp.convert_iso3_to_iso2(
    #                 df=df_result,
    #                 iso3_column='iso3_country_code',
    #                 iso2_column='iso2_country_code'
    #             )
                
    #             logger.info(f"Batch {batch_idx + 1}: Processed {len(df_result)} features via Earth Engine")
    #             return df_result
                
    #         except ee.EEException as e:
    #             backoff = min(2 ** attempt, 60)
    #             logger.warning(f"EE error in batch {batch_idx + 1} (attempt {attempt+1}), retrying in {backoff}s: {e}")
    #             time.sleep(backoff)
    #         except Exception as e:
    #             logger.error(f"General error in batch {batch_idx + 1} (attempt {attempt+1}): {e}")
    #             if attempt < MAX_RETRIES - 1:
    #                 time.sleep(2 ** attempt)
        
    #     raise RuntimeError(f"Failed to process batch {batch_idx + 1} after {MAX_RETRIES} attempts")
    

    def _process_ee_feature_collection(self, feature_collection: ee.FeatureCollection, 
                                 national_codes: Optional[List[str]], batch_idx: int) -> pd.DataFrame:
        """Process FeatureCollection using Earth Engine with retry logic"""
        
        for attempt in range(MAX_RETRIES):
            try:
                # Use whisp's existing function that handles all conversions
                df_result = whisp.whisp_stats_ee_to_df(
                    feature_collection=feature_collection,
                    external_id_column=None,
                    remove_geom=False,
                    national_codes=national_codes,
                    unit_type="ha"
                )
                
                logger.info(f"Batch {batch_idx + 1}: Processed {len(df_result)} features via Earth Engine")
                return df_result
                
            except ee.EEException as e:
                backoff = min(2 ** attempt, 60)
                logger.warning(f"EE error in batch {batch_idx + 1} (attempt {attempt+1}), retrying in {backoff}s: {e}")
                time.sleep(backoff)
            except Exception as e:
                logger.error(f"General error in batch {batch_idx + 1} (attempt {attempt+1}): {e}")
                if attempt < MAX_RETRIES - 1:
                    time.sleep(2 ** attempt)
        
        raise RuntimeError(f"Failed to process batch {batch_idx + 1} after {MAX_RETRIES} attempts")

    
# High-volume batch processor with queue management
class WhispBatchQueue:
    """Queue-based processor for very high volume scenarios"""
    
    def __init__(self, features_per_batch=50, max_concurrent_batches=10):
        self.features_per_batch = features_per_batch
        self.max_concurrent_batches = max_concurrent_batches
        self.processor = OptimizedWhispProcessor(max_concurrent_batches, features_per_batch)
        
    def process_multiple_files(self, file_paths: List[str], national_codes: Optional[List[str]] = None) -> pd.DataFrame:
        """Process multiple files with intelligent batching"""
        
        all_results = []
        total_files = len(file_paths)
        
        logger.info(f"Processing {total_files} files with {self.features_per_batch} features per EE batch")
        
        for i, file_path in enumerate(file_paths):
            logger.info(f"Processing file {i+1}/{total_files}: {file_path}")
            
            try:
                result = self.processor.process_file_optimized(file_path, national_codes)
                if not result.empty:
                    result['source_file'] = file_path
                    all_results.append(result)
                    
            except Exception as e:
                logger.error(f"Failed to process {file_path}: {e}")
        
        # Combine all results
        if all_results:
            combined = pd.concat(all_results, ignore_index=True)
            logger.info(f"Total processed: {len(combined)} features from {len(all_results)} files")
            return combined
        else:
            return pd.DataFrame()


In [4]:
folder_path = (r"C:\Users\Arnell\Downloads\processing_tests")  # Replace with your folder path

In [5]:
GEOJSON_EXAMPLE_FILEPATH = folder_path+"/random_polygons.geojson"

# Define bounds from the provided Earth Engine geometry
# area in Ghana 
bounds = [ 
    -3.04548260909834,  # min_lon
    5.253961384163733,  # min_lat
    -1.0179939534016594,  # max_lon
    7.48307210714245    # max_lat
]

# area in China
# bounds = [
#     103.44831497309737,  # min_lon
#     25.686366665187148,  # min_lat
#     109.57868606684737,  # max_lon
#     28.79200348254393    # max_lat
# ]

In [6]:
random_geojson = whisp.create_geojson(
    bounds, 
    num_polygons=10000, 
    min_area_ha=5, 
    max_area_ha=10, 
    min_number_vert=40, 
    max_number_vert=100)

GEOJSON_EXAMPLE_FILEPATH = folder_path + "/random_polygons.geojson"

import json
# Save the GeoJSON to a file
with open(GEOJSON_EXAMPLE_FILEPATH, 'w') as f:
    json.dump(random_geojson, f)

# Use example Whisp inputs (optional)
# GEOJSON_EXAMPLE_FILEPATH = whisp.get_example_data_path("geojson_example.geojson")


# Add IDs to your existing GeoJSON file

#Save to a new file (instead of overwriting)
whisp.reformat_geojson_properties(
    geojson_path=GEOJSON_EXAMPLE_FILEPATH, 
    id_field="internal_id",
    output_path=folder_path + "/random_polygons_with_ids.geojson",
    remove_properties=True
)

2025-06-17 12:40:39,476 - INFO - Created 10,000 records


Added internal_id to GeoJSON and saved to C:\Users\Arnell\Downloads\processing_tests/random_polygons_with_ids.geojson


In [17]:

# Example usage with controlled batch sizes
if __name__ == "__main__":
    
    # Configure batch size based on your data characteristics
    FEATURES_PER_EE_REQUEST = 25  # Small batches for complex geometries
    MAX_CONCURRENT_EE_REQUESTS = 20  # Conservative for quota management
    
    # Initialize processor
    processor = OptimizedWhispProcessor(
        max_concurrent=MAX_CONCURRENT_EE_REQUESTS,
        features_per_batch=FEATURES_PER_EE_REQUEST
    )
    
    # Process file with controlled batching
    try:
        # GEOJSON_EXAMPLE_FILEPATH = whisp.get_example_data_path("geojson_example.geojson")
        
        logger.info(f"Processing with {FEATURES_PER_EE_REQUEST} features per Earth Engine request")
        logger.info(f"Maximum {MAX_CONCURRENT_EE_REQUESTS} concurrent requests")
        
        result_df = processor.process_file_optimized(
            GEOJSON_EXAMPLE_FILEPATH, 
            national_codes=["br", "co"]
        )
        
        if not result_df.empty:
            print(f"Success! Processed {len(result_df)} features")
            print("\nFirst 5 rows:")
            print(result_df.head())
            
            # Save results
            result_df.to_csv("optimized_whisp_results.csv", index=False)
            logger.info("Results saved to optimized_whisp_results.csv")
        else:
            print("No results produced")
            
        print(f"Processing stats: {processor.processing_stats}")
        
    except Exception as e:
        logger.error(f"Processing failed: {e}")

2025-06-17 12:01:55,075 - INFO - Processing with 25 features per Earth Engine request
2025-06-17 12:01:55,076 - INFO - Maximum 20 concurrent requests
2025-06-17 12:01:59,913 - INFO - Processing 10000 features in batches of 25
2025-06-17 12:01:59,932 - INFO - Created 400 batches for Earth Engine processing
2025-06-17 12:02:02,520 - WARNING - Connection pool is full, discarding connection: earthengine-highvolume.googleapis.com. Connection pool size: 10
2025-06-17 12:02:02,614 - WARNING - Connection pool is full, discarding connection: earthengine-highvolume.googleapis.com. Connection pool size: 9


['Area', 'Oil_palm_Descals', 'European_Primary_Forest', 'ESA_fire_before_2020', 'ESA_fire_2001', 'ESA_fire_2002', 'ESA_fire_2003', 'ESA_fire_2004', 'ESA_fire_2005', 'ESA_fire_2006', 'ESA_fire_2007', 'ESA_fire_2008', 'ESA_fire_2009', 'ESA_fire_2010', 'ESA_fire_2011', 'ESA_fire_2012', 'ESA_fire_2013', 'ESA_fire_2014', 'ESA_fire_2015', 'ESA_fire_2016', 'ESA_fire_2017', 'ESA_fire_2018', 'ESA_fire_2019', 'ESA_fire_2020', 'ESA_TC_2020', 'ESRI_2023_crop', 'ESRI_2023_TC', 'Cocoa_ETH', 'Cocoa_2023_FDaP', 'Cocoa_FDaP', 'Forest_FDaP', 'Oil_palm_2023_FDaP', 'Oil_palm_FDaP', 'Rubber_2023_FDaP', 'Rubber_FDaP', 'GFT_naturally_regenerating', 'GFT_planted_plantation', 'GFT_primary', 'GFC_TC_2020', 'GFC_loss_after_2020', 'GFC_loss_before_2020', 'GFC_loss_year_2001', 'GFC_loss_year_2002', 'GFC_loss_year_2003', 'GFC_loss_year_2004', 'GFC_loss_year_2005', 'GFC_loss_year_2006', 'GFC_loss_year_2007', 'GFC_loss_year_2008', 'GFC_loss_year_2009', 'GFC_loss_year_2010', 'GFC_loss_year_2011', 'GFC_loss_year_2012',

2025-06-17 12:02:05,556 - WARNING - Connection pool is full, discarding connection: earthengine-highvolume.googleapis.com. Connection pool size: 10


['Area', 'Oil_palm_Descals', 'European_Primary_Forest', 'ESA_fire_before_2020', 'ESA_fire_2001', 'ESA_fire_2002', 'ESA_fire_2003', 'ESA_fire_2004', 'ESA_fire_2005', 'ESA_fire_2006', 'ESA_fire_2007', 'ESA_fire_2008', 'ESA_fire_2009', 'ESA_fire_2010', 'ESA_fire_2011', 'ESA_fire_2012', 'ESA_fire_2013', 'ESA_fire_2014', 'ESA_fire_2015', 'ESA_fire_2016', 'ESA_fire_2017', 'ESA_fire_2018', 'ESA_fire_2019', 'ESA_fire_2020', 'ESA_TC_2020', 'ESRI_2023_crop', 'ESRI_2023_TC', 'Cocoa_ETH', 'Cocoa_2023_FDaP', 'Cocoa_FDaP', 'Forest_FDaP', 'Oil_palm_2023_FDaP', 'Oil_palm_FDaP', 'Rubber_2023_FDaP', 'Rubber_FDaP', 'GFT_naturally_regenerating', 'GFT_planted_plantation', 'GFT_primary', 'GFC_TC_2020', 'GFC_loss_after_2020', 'GFC_loss_before_2020', 'GFC_loss_year_2001', 'GFC_loss_year_2002', 'GFC_loss_year_2003', 'GFC_loss_year_2004', 'GFC_loss_year_2005', 'GFC_loss_year_2006', 'GFC_loss_year_2007', 'GFC_loss_year_2008', 'GFC_loss_year_2009', 'GFC_loss_year_2010', 'GFC_loss_year_2011', 'GFC_loss_year_2012',

2025-06-17 12:02:05,774 - WARNING - Connection pool is full, discarding connection: earthengine-highvolume.googleapis.com. Connection pool size: 10


...

['Area', 'Oil_palm_Descals', 'European_Primary_Forest', 'ESA_fire_before_2020', 'ESA_fire_2001', 'ESA_fire_2002', 'ESA_fire_2003', 'ESA_fire_2004', 'ESA_fire_2005', 'ESA_fire_2006', 'ESA_fire_2007', 'ESA_fire_2008', 'ESA_fire_2009', 'ESA_fire_2010', 'ESA_fire_2011', 'ESA_fire_2012', 'ESA_fire_2013', 'ESA_fire_2014', 'ESA_fire_2015', 'ESA_fire_2016', 'ESA_fire_2017', 'ESA_fire_2018', 'ESA_fire_2019', 'ESA_fire_2020', 'ESA_TC_2020', 'ESRI_2023_crop', 'ESRI_2023_TC', 'Cocoa_ETH', 'Cocoa_2023_FDaP', 'Cocoa_FDaP', 'Forest_FDaP', 'Oil_palm_2023_FDaP', 'Oil_palm_FDaP', 'Rubber_2023_FDaP', 'Rubber_FDaP', 'GFT_naturally_regenerating', 'GFT_planted_plantation', 'GFT_primary', 'GFC_TC_2020', 'GFC_loss_after_2020', 'GFC_loss_before_2020', 'GFC_loss_year_2001', 'GFC_loss_year_2002', 'GFC_loss_year_2003', 'GFC_loss_year_2004', 'GFC_loss_year_2005', 'GFC_loss_year_2006', 'GFC_loss_year_2007', 'GFC_loss_year_2008', 'GFC_loss_year_2009', 'GFC_loss_year_2010', 'GFC_loss_year_2011', 'GFC_loss_year_2012',

2025-06-17 12:17:45,745 - INFO - Batch 385: Processed 25 features via Earth Engine
2025-06-17 12:17:45,777 - INFO - Completed batch 385/400
2025-06-17 12:17:46,446 - INFO - Batch 383: Processed 25 features via Earth Engine
2025-06-17 12:17:46,449 - INFO - Completed batch 383/400
2025-06-17 12:17:46,666 - INFO - Batch 380: Processed 25 features via Earth Engine
2025-06-17 12:17:46,666 - INFO - Completed batch 380/400
2025-06-17 12:17:48,449 - INFO - Batch 379: Processed 25 features via Earth Engine
2025-06-17 12:17:48,465 - INFO - Completed batch 379/400
2025-06-17 12:17:50,337 - INFO - Batch 388: Processed 25 features via Earth Engine
2025-06-17 12:17:50,398 - INFO - Completed batch 388/400
2025-06-17 12:17:50,895 - INFO - Batch 386: Processed 25 features via Earth Engine
2025-06-17 12:17:50,931 - INFO - Completed batch 386/400
2025-06-17 12:17:51,323 - INFO - Batch 381: Processed 25 features via Earth Engine
2025-06-17 12:17:51,323 - INFO - Completed batch 381/400
2025-06-17 12:17:51,

Success! Processed 10000 features

First 5 rows:
                                                 geo   Admin_Level_1    Area  \
0  {'type': 'Polygon', 'coordinates': [[[-1.21205...  Ashanti Region   7.438   
1  {'type': 'Polygon', 'coordinates': [[[-1.05199...  Ashanti Region   6.608   
2  {'type': 'Polygon', 'coordinates': [[[-1.69089...  Ashanti Region   9.694   
3  {'type': 'Polygon', 'coordinates': [[[-1.94252...  Ashanti Region   9.026   
4  {'type': 'Polygon', 'coordinates': [[[-2.29184...     Bono Region  11.246   

   Centroid_lat  Centroid_lon  Cocoa_2023_FDaP  Cocoa_ETH  Cocoa_FDaP Country  \
0      6.095634     -1.210289            0.685      0.071       0.040     GHA   
1      6.574340     -1.050201            2.056      5.739       1.439     GHA   
2      6.209931     -1.689022            0.000      0.000       0.000     GHA   
3      6.863018     -1.940482            3.534      9.026       3.806     GHA   
4      7.442607     -2.289691            0.521      0.000       0

2025-06-17 12:18:08,329 - INFO - Results saved to optimized_whisp_results.csv


Processing stats: {'completed': 400, 'failed': 0, 'total': 0}


In [15]:
result_df  # Display first few rows of combined results

,geo,Admin_Level_1,Area,Centroid_lat,Centroid_lon,Cocoa_2023_FDaP,Cocoa_ETH,Cocoa_FDaP,Country,ESA_TC_2020,...,nBR_MapBiomas_col9_pasture_2020,nBR_MapBiomas_col9_pc_2020,nBR_MapBiomas_col9_silviculture_Brazil_2020,nBR_MapBiomas_col9_soy_2020,nBR_PRODES_deforestation_Brazil_post2020,nBR_PRODES_deforestation_Brazil_upto2020,nCO_ideam_agroforest_2020,nCO_ideam_forest_2020,plotId,ProducerCountry
0,"{'type': 'Polygon', 'coordinates': [[[-1.36827...",Ashanti Region,8.084,7.137982,-1.366372,0.000,0.000,0.000,GHA,3.717,...,0,0,0,0,0,0,0,0,1,GH
1,"{'type': 'Polygon', 'coordinates': [[[-2.52667...",Western North Region,8.193,6.098759,-2.524697,0.000,0.000,0.000,GHA,8.193,...,0,0,0,0,0,0,0,0,2,GH
2,"{'type': 'Polygon', 'coordinates': [[[-1.17849...",Eastern Region,9.541,6.033595,-1.176477,6.268,9.480,5.703,GHA,9.486,...,0,0,0,0,0,0,0,0,3,GH
3,"{'type': 'Polygon', 'coordinates': [[[-2.14242...",Western Region,7.606,5.688219,-2.140685,0.015,2.066,0.040,GHA,5.787,...,0,0,0,0,0,0,0,0,4,GH
4,"{'type': 'Polygon', 'coordinates': [[[-2.09582...",Ashanti Region,8.706,6.985618,-2.093931,6.937,8.696,7.679,GHA,2.534,...,0,0,0,0,0,0,0,0,5,GH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,"{'type': 'Polygon', 'coordinates': [[[-1.48174...",Ashanti Region,10.449,6.851505,-1.479588,0.000,0.000,0.000,GHA,7.111,...,0,0,0,0,0,0,0,0,21,GH
996,"{'type': 'Polygon', 'coordinates': [[[-1.38644...",Ashanti Region,8.994,6.112829,-1.384571,0.469,8.332,0.366,GHA,8.994,...,0,0,0,0,0,0,0,0,22,GH
997,"{'type': 'Polygon', 'coordinates': [[[-1.68425...",Ashanti Region,6.037,6.821568,-1.682674,0.000,0.000,0.000,GHA,0.505,...,0,0,0,0,0,0,0,0,23,GH
998,"{'type': 'Polygon', 'coordinates': [[[-1.03219...",Eastern Region,7.255,6.439917,-1.030592,3.013,7.153,1.258,GHA,6.665,...,0,0,0,0,0,0,0,0,24,GH


Classic Whisp

In [1]:
# Earth Engine and Common Libraries
import ee
from pathlib import Path

# Authenticate and initialize Earth Engine
try:
    ee.Initialize()  # Try to use existing credentials first
except Exception:
    ee.Authenticate()
    ee.Initialize()

In [2]:
import openforis_whisp as whisp


In [ ]:
whisp = whisp.whisp_formatted_stats_geojson_to_df(GEOJSON_EXAMPLE_FILEPATH)

Reading GeoJSON file from: C:\Users\Arnell\Downloads\processing_tests\random_polygons.geojson


c:\Users\Arnell\Documents\GitHub\whisp\.venv\Lib\site-packages\ee\deprecation.py:207: DeprecationWarning: 

Attention required for projects/forestdatapartnership/assets/cocoa/model_2024a! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/projects_forestdatapartnership_assets_cocoa_model_2024a

  warnings.warn(warning, category=DeprecationWarning)
c:\Users\Arnell\Documents\GitHub\whisp\.venv\Lib\site-packages\ee\deprecation.py:207: DeprecationWarning: 

Attention required for projects/forestdatapartnership/assets/palm/model_2024a! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/projects_forestdatapartnership_assets_palm_model_2024a

  warnings.warn(warning, category=DeprecationWarning)
c:\Users\Arnell\Documents\GitHub\whisp\.venv\Lib\site-packages\ee\deprecation.py

['Area', 'Oil_palm_Descals', 'European_Primary_Forest', 'ESA_fire_before_2020', 'ESA_fire_2001', 'ESA_fire_2002', 'ESA_fire_2003', 'ESA_fire_2004', 'ESA_fire_2005', 'ESA_fire_2006', 'ESA_fire_2007', 'ESA_fire_2008', 'ESA_fire_2009', 'ESA_fire_2010', 'ESA_fire_2011', 'ESA_fire_2012', 'ESA_fire_2013', 'ESA_fire_2014', 'ESA_fire_2015', 'ESA_fire_2016', 'ESA_fire_2017', 'ESA_fire_2018', 'ESA_fire_2019', 'ESA_fire_2020', 'ESA_TC_2020', 'ESRI_2023_crop', 'ESRI_2023_TC', 'Cocoa_ETH', 'Cocoa_2023_FDaP', 'Cocoa_FDaP', 'Forest_FDaP', 'Oil_palm_2023_FDaP', 'Oil_palm_FDaP', 'Rubber_2023_FDaP', 'Rubber_FDaP', 'GFT_naturally_regenerating', 'GFT_planted_plantation', 'GFT_primary', 'GFC_TC_2020', 'GFC_loss_after_2020', 'GFC_loss_before_2020', 'GFC_loss_year_2001', 'GFC_loss_year_2002', 'GFC_loss_year_2003', 'GFC_loss_year_2004', 'GFC_loss_year_2005', 'GFC_loss_year_2006', 'GFC_loss_year_2007', 'GFC_loss_year_2008', 'GFC_loss_year_2009', 'GFC_loss_year_2010', 'GFC_loss_year_2011', 'GFC_loss_year_2012',